# Part 1: Financial Metrics - Interactive Notebook

This notebook accompanies Part 1 of the Puffin tutorial. Practice calculating financial metrics with real market data.

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

# Fetch sample data
spy = yf.download('SPY', start='2020-01-01', end='2024-01-01')
prices = spy['Close']
print(f"Loaded {len(prices)} days of SPY data")
prices.plot(title='SPY Price', figsize=(12, 4))
plt.show()

In [ ]:
# Calculate daily returns
returns = prices.pct_change().dropna()

print(f"Mean daily return: {returns.mean():.4%}")
print(f"Daily volatility: {returns.std():.4%}")
print(f"Annualized volatility: {returns.std() * np.sqrt(252):.4%}")

returns.hist(bins=50, figsize=(10, 4))
plt.title('Distribution of Daily Returns')
plt.xlabel('Return')
plt.show()

In [ ]:
# Sharpe Ratio
risk_free_rate = 0.05
excess_returns = returns - risk_free_rate / 252
sharpe = (excess_returns.mean() / excess_returns.std()) * np.sqrt(252)
print(f"Sharpe Ratio: {sharpe:.2f}")

In [ ]:
# Drawdown analysis
equity = (1 + returns).cumprod()
peak = equity.cummax()
drawdown = (equity - peak) / peak

print(f"Maximum Drawdown: {drawdown.min():.2%}")

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 6), sharex=True)
equity.plot(ax=ax1, title='Equity Curve')
drawdown.plot(ax=ax2, title='Drawdown', color='red')
ax2.fill_between(drawdown.index, drawdown.values, 0, alpha=0.3, color='red')
plt.tight_layout()
plt.show()

## Exercises

1. Calculate the Sharpe ratio for QQQ and IWM over the same period. Which performed better on a risk-adjusted basis?
2. Find the date of the maximum drawdown. What market event caused it?
3. Calculate the rolling 60-day Sharpe ratio. How does it change over time?